In [1]:
%reload_ext autoreload
%autoreload 2

import os
import shutil
import numpy as np
from scipy import stats

import py21cmfish
from py21cmfish.power_spectra import *
from py21cmfish.io import *

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

/n/home07/yitians/.conda/envs/dm21cm/lib/python3.11/site-packages/py21cmfast/_cfg.py:58: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn(
/n/home07/yitians/.conda/envs/dm21cm/lib/python3.11/site-packages/py21cmfast/_cfg.py:42: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn("Your configuration file is out of date. Updating...")


## 1. 21cmFISH standard astrophysics

In [ ]:
noise_dir = '/n/home07/yitians/dm21cm/21cmSense_fid_EOS21/'
data_dir = '/n/holylabs/LABS/iaifi_lab/Users/yitians/dm21cm/outputs/bkg/LightCones/'


astro_params_vary = ['DM', 'F_STAR10', 'F_STAR7_MINI', 'ALPHA_STAR', 'ALPHA_STAR_MINI', 't_STAR',
                     'F_ESC10', 'F_ESC7_MINI', 'ALPHA_ESC', 'L_X', 'L_X_MINI', 'NU_X_THRESH', 'A_LW']
default_param_values = [0, -1.25, -2.5, 0.5, 0.0, 0.5, -1.35, -1.35, -0.3, 40.5, 40.5, 500, 2.0]

astro_params_vary = list(np.array(astro_params_vary))
default_param_values = default_param_values
astro_params_fid = dict()
for i in range(len(astro_params_vary)):
    astro_params_fid[astro_params_vary[i]] = default_param_values[i]
    
# Load each parameter into a dictionary
params_EoS = {}

for param in astro_params_vary[1:]:
    params_EoS[param] = py21cmfish.Parameter(
        HII_DIM=128, BOX_LEN=256, param=param,
        output_dir = data_dir,
        PS_err_dir = noise_dir,
        new = False
)

In [ ]:
params_EoS['F_STAR10'].PS

## 2. Fisher analysis at each mass

In [4]:
mass_s = 10**np.array([15, 16, 17, 18], dtype=float)
inj_s = np.array([1e-10, 3e-7, 1e-2, 1e0])

In [6]:
inj_dir = '/n/holylabs/LABS/iaifi_lab/Users/yitians/dm21cm/outputs/pbh'
os.listdir(inj_dir)

['Mass_0', 'Mass_3', 'Mass_2', 'Mass_1']

In [ ]:
# Gotta put the fiducial in each mass directory
for i in range(len(mass_s)):
    print(i)
    shutil.copyfile(f'{data_dir}/LightCone_z5.0_HIIDIM=128_BOXLEN=256_fisher_fid_r54321.h5',
                    f'{inj_dir}/Mass_'+str(i) + '/LightCones/LightCone_z5.0_HIIDIM=128_BOXLEN=256_fisher_fid_r54321.h5')

In [ ]:
f_inv_s = []

for mass_index in np.arange(len(mass_s)):
    
    for param in astro_params_vary[:1]:
        params_EoS[param] = py21cmfish.Parameter(HII_DIM=128, BOX_LEN=256, param=param,
                                              output_dir=f'{inj_dir}/Mass_'+str(mass_index)+'/LightCones/',
                                              PS_err_dir=noise_dir, new=False)

    
    Fij_matrix_PS, Finv_PS= py21cmfish.make_fisher_matrix(params_EoS,
                                                   fisher_params=astro_params_vary,
                                                   hpeak=0.0, obs='PS',
                                                   k_min=0.1, k_max=1,
                                                   sigma_mod_frac=0.2,
                                                   add_sigma_poisson=True)
    
    f_inv_s.append(Finv_PS[0, 0])
sigma_s = np.sqrt(f_inv_s)

In [15]:
np.savetxt('../outputs/limits/pbh_limits.txt', np.array([mass_s, inj_s, sigma_s]).T, header='mass_s inj_s sigma_s')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

one_sigma = inj_s * sigma_s
upper_limit = np.sqrt(stats.chi2.ppf(.9, df=1)) * one_sigma

ax.plot(mass_s, upper_limit, 'k')
ax.fill_between(mass_s, upper_limit + one_sigma, upper_limit - one_sigma, color=mpl.colormaps['viridis'](0.75))
ax.fill_between(mass_s, upper_limit + 2*one_sigma, upper_limit + one_sigma, color=mpl.colormaps['viridis'](1.0))

ax.set(xscale='log', yscale='log')
ax.set(xlabel=r'$M$ [g]', ylabel=r'$f$')